#  Bypass ApkID check

In [ ]:
# Copyright (C) 2019  Communication Network Security Laboratory, SSU, Seoul

# This program is free software; you can redistribute it and/or modify
# it under the terms of the GNU General Public License as published by
# the Free Software Foundation; either version 2 of the License, or
# (at your option) any later version.

# This program is distributed in the hope that it will be useful,
# but WITHOUT ANY WARRANTY; without even the implied warranty of
# MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.  See the
# GNU General Public License for more details.

In [157]:
import os
import os.path
import json
import re
import shutil

DEBUG_FLG = 0

def _exec(cmd, silent=False):
    """Execute shell"""
    import subprocess
    global DEBUG_FLG
    if DEBUG_FLG == 1:
        logger.info("cmd: %s" % cmd)

    channel = open(os.devnull, 'wb') if silent is True else subprocess.PIPE
    p = subprocess.Popen(cmd, stdout=channel, stderr=channel, shell=True)
    out, err = p.communicate()
    if err:
        print(err)
    return out.decode("utf-8").strip()


class BypassModel(object):
    """Model for Bypass packer."""
    def __init__(self, apk_file=None):
        self.UNPACK_DIR="/tmp/unpack"
        self.OUTPUT_DIR = "bypass_apkid"
        if apk_file:
            self.setFile(apk_file)
            
    def setFile(self, apk_path, apk_file):
        """Set target apk file"""
        self.apk_full = os.path.join(apk_path, apk_file)
        self.apk_file = apk_file
        self.apk_path = apk_path
    
    def getPackerDict(self):
        """Get packer dictionary object."""
        if not os.path.isfile("dict/packer_dict.json"):
            packer_dict = self.convertYara()
        else:
            with open("dict/packer_dict.json") as f:
                packer_dict = json.load(f)
        return packer_dict
    
    def _unpack(self):
        if os.path.isdir(self.UNPACK_DIR):
           shutil.rmtree(self.UNPACK_DIR)
        os.mkdir(self.UNPACK_DIR)
        # Unzip 
        cmd = "unzip %s -d %s" % (self.apk_full, self.UNPACK_DIR)
        _exec(cmd)
        print("Unpack done!!")
        
         # Get trace info
    def _findTrace(self, trace):
        """Get traces from extracted folder."""
        traces = []
        # CHECK if it is path
        if trace[-1] == "/":
            cmd = "find %s -path \"*%s\" -print" % (self.UNPACK_DIR, trace[:-1])
        else:
            cmd = "find %s | grep \"%s\"" % (self.UNPACK_DIR, trace)
        try:
            out = _exec(cmd)
            if out:
                for line in out.split("\n"):
                    traces.append(line)
        except Exception as ex:
            self._log("Cannot find traces. Error: %s" % ex, 1)
        return traces
    
    def _modifyTrace(self, trace):
        cmd = "file %s" % trace
        out = _exec(cmd)
        value = out.split(":")[0].strip()
        check = out.split(":")[-1].strip()
        name, ext = os.path.splitext(os.path.basename(value))
        path, filename = os.path.split(value)
        out_str = ""
        obfFlg = 0
        for c in name:
            if obfFlg == 0:
                out_str += "%d" % ord(c) 
                obfFlg = 1
            else:
                out_str += c
                obfFlg = 0
        if "directory" not in check:
            # Add extension back
            if out_str[-1] != ".":
                out_str += "."
            out_str += ext
        changed_path = os.path.join(path, out_str)
        cmd = "mv %s %s" % (value, changed_path)
        print(cmd)
        _exec(cmd)
        return changed_path
                    
    def _pack(self):
        if not os.path.isdir(self.OUTPUT_DIR):
            os.mkdir(self.OUTPUT_DIR)
        output_zip = os.path.join(self.OUTPUT_DIR, self.apk_file)
        if os.path.isfile(output_zip):
            cmd = "rm -f %s" % output_zip
            _exec(cmd)
        cmd = "zip -r %s %s" % (output_zip, self.UNPACK_DIR)
        _exec(cmd)
        print("Done packing!!")
        return
        
    def byPass(self):
        # Unpack apk
        self._unpack()
        packer_dict = self.getPackerDict()
        traces_list = self._getTraces(packer_dict)
        print("Got traces: %s" %traces_list)
        print(traces_list)
        for packer, traceArray in traces_list.items():
            for trace in traceArray:
                encoded_trace = self._modifyTrace(trace[0])
                print("Trace: %s, Encoded: %s"  % (trace, encoded_trace))
        self._pack()
        
    def logApkIdOutput(self):
        cmd = "apkid %s >> %s/failed_bypass.txt" % (self.UNPACK_DIR, self.OUTPUT_DIR)
        _exec(cmd)
        return
        
         # Get traces of a packer
    def _getTraces(self, packer_dict):
        """Get traces of packers."""
        traces_list = {}
        for name, traces in packer_dict.items():
            # more than one
            if ";" in traces:
                for trace in traces.split(";"):
                    output = self._findTrace(trace)
                    if output:
                        if name in traces_list:
                            traces_list[name].append(output)
                        else:
                            traces_list[name] = []
                            traces_list[name].append(output)
            else:
                output = self._findTrace(traces)
                if output:
                    # contain traces
                        if name in traces_list:
                            traces_list[name].append(output)
                        else:
                            traces_list[name] = []
                            traces_list[name].append(output)      
        return traces_list

    def yaraToDict(self, lines):
        """Convert yara file to dictionary."""
        # Convert packers.yara
        openFlg = 0
        strFlg = 0
        old_value = None
        new_value = None
        packer_dict = {}
        packer_name = None
        for content in lines:
            if content:
                if re.match(r'^rule', content):
                    # Begin of rule
                    openFlg = 1
                    packer_name = content.split(" ")[1]
                if openFlg and "strings" in content:
                    # Start string
                    strFlg = 1
                if "condition" in content:
                    openFlg = 0
                    strFlg = 0
                    old_value = None
                    new_value = None
                if openFlg and strFlg and "strings" not in content:
                    # get traces
                    if content and "=" in content:
                        find_value = re.findall(r'"([^"]*)"', content)
                        if not find_value:
                            value = content.split("=")[1].replace("\"",
                                                                  "").strip()
                        else:
                            value = find_value[0].strip()

                        if packer_name not in packer_dict:
                            packer_dict[packer_name] = value
                        else:
                            old_value = packer_dict[packer_name]
                            new_value = "%s;%s" % (old_value, value)
                            packer_dict[packer_name] = new_value
        return packer_dict
    
    def convertYara(self):
        """Get and convert yara files. Please check the github and get this one."""
        yara_url = [
            'https://raw.githubusercontent.com/rednaga/APKiD/master/apkid/rules/apk/packers.yara',
            'https://raw.githubusercontent.com/rednaga/APKiD/master/apkid/rules/apk/common.yara'
        ]
        packer_dict = {}
        if not os.path.isdir("yara"):
            os.mkdir("yara")
        # Download yara rules
        for url in yara_url:
            if not os.path.isfile("yara/%s" % url.split("/")[-1]):
                cmd = "wget %s -O yara/%s" % (url, url.split("/")[-1])
                _exec(cmd)
        try:
            with open("yara/packers.yara") as f:
                lines = f.readlines()
            packer_dict = self.yaraToDict(lines)
        except Exception as ex:
            self._log("Error: %s" % ex, 1)
        # Write dict to file
        try:
            if not os.path.isdir("dict"):
                os.mkdir("dict")
            with open("dict/packer_dict.json", 'w') as f:
                json.dump(packer_dict, f)
        except Exception as ex:
            self._log("Error: %s " % ex, 1)
        return packer_dict
    
    def _log(self, string, t):
        if DEBUG_FLG:
            if t == 0:
                print("[+] %s" % string)
            if t == 1:
                print("[-] %s" % string)
            if t == 2:
                print("[!] %s" % string)
        return
        
class ApkModel(object):
    """Model for APK file."""
    def __init__(self, apk_path, apk_file):
        self.apk = apk_file
        self.path = apk_path
        
    def packingInfo(self):
        """Check packer exist."""
        cmd = "apkid %s" % os.path.join(self.path, self.apk)
        out = _exec(cmd)
        return out

def main():
    """Main function."""
    ##
    #  CODE FOR BYPASSING
    ##
    
    # PACKER_FOLDER = "packers"
    # bypass_model = BypassModel()
    # for f in os.listdir(PACKER_FOLDER):
    #    # Make model
    #    print("Modifying %s" % f)
    #    full_path = os.path.join(PACKER_FOLDER, f)
    #    model = ApkModel(PACKER_FOLDER, f)
    #    out = model.packingInfo()
    #    bypass_model.setFile(PACKER_FOLDER, f)
    #    bypass_model.byPass()
    
    ##
    #  CODE FOR CHECKING BYPASS RESULT
    #  WE OUTPUTED ALL PACKER DETECTED INTO FILE CALLED: failed_bypass.txt  
    ##
    # get failed 
    cmd = "cat failed_bypass.txt | grep apk"
    out = _exec(cmd)
    PACKER_FOLDER = "packers"
    bypass_model = BypassModel()
    for line in out.split("\n"):
        file = line.split(" ")[-1]
        f = os.path.basename(file)
        print("Modifying %s" % f)
        full_path = os.path.join(PACKER_FOLDER, f)
        model = ApkModel(PACKER_FOLDER, f)        
        bypass_model.setFile(PACKER_FOLDER, f)
        bypass_model.byPass()    
        bypass_model.logApkIdOutput()
      
if (__name__ == "__main__"):
    main()

Modifying 04717bd65e06dab0749b011afae55efa.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 078ccae070b94064aa75535ecc061e46.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 0948cee6be6c8b2fa3955ef6d4a3a1dc.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 0a222de053c438eb3449a01fd49d7458.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 0cdda8b42139874000d2918262041705.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 10ef7e182fabce89627d410246cbd8e1.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 26a414131320a246255e05c3475b6f76.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 2b791c1239be130a2413de9e6d3b294f.apk
Unpack done!!
Got traces: {}
{}
Done packing!!
Modifying 2efab2f276712af4342e60af9f6cc7cf.apk
Unpack done!!
Got traces: {'ijiami': [['/tmp/unpack/assets/ijiami.dat'], ['/tmp/unpack/assets/ijm_lib']], 'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecr

Done packing!!
Modifying 998d8131f4c07fb7cd79cfb66476a08e.apk
Unpack done!!
Got traces: {'bangcle': [['/tmp/unpack/lib/armeabi/libsecexe.so', '/tmp/unpack/lib/armeabi-v7a/libsecexe.so'], ['/tmp/unpack/lib/armeabi/libsecmain.so', '/tmp/unpack/lib/armeabi-v7a/libsecmain.so'], ['/tmp/unpack/assets/bangcle_classes.jar']], 'unicom_loader': [['/tmp/unpack/assets/bangcle_classes.jar']]}
{'bangcle': [['/tmp/unpack/lib/armeabi/libsecexe.so', '/tmp/unpack/lib/armeabi-v7a/libsecexe.so'], ['/tmp/unpack/lib/armeabi/libsecmain.so', '/tmp/unpack/lib/armeabi-v7a/libsecmain.so'], ['/tmp/unpack/assets/bangcle_classes.jar']], 'unicom_loader': [['/tmp/unpack/assets/bangcle_classes.jar']]}
mv /tmp/unpack/lib/armeabi/libsecexe.so /tmp/unpack/lib/armeabi/108i98s101c101x101..so
Trace: ['/tmp/unpack/lib/armeabi/libsecexe.so', '/tmp/unpack/lib/armeabi-v7a/libsecexe.so'], Encoded: /tmp/unpack/lib/armeabi/108i98s101c101x101..so
mv /tmp/unpack/lib/armeabi/libsecmain.so /tmp/unpack/lib/armeabi/108i98s101c109a105n..

Got traces: {'tencent': [['/tmp/unpack/lib/armeabi/mix.dex']], 'apkprotect': [['/tmp/unpack/apkprotect.com/key.dat'], ['/tmp/unpack/apkprotect.com']]}
{'tencent': [['/tmp/unpack/lib/armeabi/mix.dex']], 'apkprotect': [['/tmp/unpack/apkprotect.com/key.dat'], ['/tmp/unpack/apkprotect.com']]}
mv /tmp/unpack/lib/armeabi/mix.dex /tmp/unpack/lib/armeabi/109i120..dex
Trace: ['/tmp/unpack/lib/armeabi/mix.dex'], Encoded: /tmp/unpack/lib/armeabi/109i120..dex
mv /tmp/unpack/apkprotect.com/key.dat /tmp/unpack/apkprotect.com/107e121..dat
Trace: ['/tmp/unpack/apkprotect.com/key.dat'], Encoded: /tmp/unpack/apkprotect.com/107e121..dat
mv /tmp/unpack/apkprotect.com /tmp/unpack/97p107p114o116e99t
Trace: ['/tmp/unpack/apkprotect.com'], Encoded: /tmp/unpack/97p107p114o116e99t
Done packing!!
Modifying eb7e9a04175b59099c2f2d1e7aa6498c.apk
b'warning:  stripped absolute path spec from /0\nwarning:  stripped absolute path spec from /0.aac\nwarning:  stripped absolute path spec from /0.xml\nwarning:  stripped ab

Done packing!!
Modifying VirusShare_30fb67f5665f5ff6e8f850ecccd40eb1.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack

Done packing!!
Modifying VirusShare_6a2ce8451738cbd324f4036fe8e45bfb.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack

Done packing!!
Modifying VirusShare_9cdb5150fe3cc7a6108b9b22a2fe2513.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack

Done packing!!
Modifying VirusShare_bd6c22fbc2cf102389a99adada958697.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack

Done packing!!
Modifying VirusShare_dc3257fc65de29e25e755d5714f90210.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/arm64-v8a/libdecrypt.jar', '/tmp/unpack/assets/armeabi-v7a/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/arm64-v8a/libunicomsdk.jar', '/tmp/unpack/assets/armeabi-v7a/libunicomsdk.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack

Done packing!!
Modifying VirusShare_fdc95cb3b7c9a81229fce494ac0af40f.apk
Unpack done!!
Got traces: {'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/mips/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/mips/libunicomsdk.jar'], ['/tmp/unpack/assets/classes.jar']]}
{'unicom_loader': [['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/mips/libdecrypt.jar'], ['/tmp/unpack/assets/x86/libunicomsdk.jar', '/tmp/unpack/assets/armeabi/libunicomsdk.jar', '/tmp/unpack/assets/mips/libunicomsdk.jar'], ['/tmp/unpack/assets/classes.jar']]}
mv /tmp/unpack/assets/x86/libdecrypt.jar /tmp/unpack/assets/x86/108i98d101c114y112t..jar
Trace: ['/tmp/unpack/assets/x86/libdecrypt.jar', '/tmp/unpack/assets/armeabi/libdecrypt.jar', '/tmp/unpack/assets/mips/libdecrypt.jar'], Encoded: /tmp/unpack/assets/x86